In [17]:
# Le but de cette partie est de créer un reseau de neuronnes
# Qui va tenter de prédire les valeurs d'achat et de vente des
# Traders générés aléatoirement - qui fotn quand même +2% par mois -
# A partir du stock marcket des 30 derniers jours
# Enfin, on va simuler les achats/ventes du réseau de neuronne sur dix ans
# Et regarder ensuite si son portefeuille a bien évolué

In [18]:
import tensorflow.keras.layers as lyrs
import tensorflow.keras.models as mod
import sklearn.preprocessing as prepro
import numpy as np
import matplotlib.pyplot as plt
import copy

In [5]:
def load_trader():
    f = open('trader_data\\trader0.csv', 'rb').readlines()[1:]
    prices = []
    for b_line in f:
        line = b_line.decode("utf-8")
        price = float(line.split(',')[4])
        prices.append(price)
    ratios = []
    for i in range(0,50):
        f = open('trader_data\\trader'+ str(i) +'.csv', 'rb').readlines()[1:]
        tmp = []
        for b_line in f:
            line = b_line.decode("utf-8")
            ratio = float(line.split(',')[3])
            tmp.append(ratio)
        ratios.append(tmp)
    return prices, ratios

In [8]:
prices, ratios = load_trader()
prices = np.array(prices)
ratios = np.array(ratios)
length = len(ratios[0])
TRAIN_SIZE = int(length*0.8)

In [4]:
train = []
test = []
train_target = []
test_target = []

In [9]:
for i in range(30, length):
    if i < TRAIN_SIZE:
        train.append(prices[i-30:i])
        train_target.append(ratios[0][i])
    else:
        test.append(prices[i-30:i])
        test_target.append(ratios[0][i])

In [10]:
train = np.array(train)
train_target = np.array(train_target)
test = np.array(test)
test_target = np.array(test_target)

In [11]:
model = mod.Sequential()
model.add(lyrs.Dense(1000, input_shape = (30,)))
model.add(lyrs.Activation('tanh'))
model.add(lyrs.Dropout(0.25))
model.add(lyrs.Dense(500))
model.add(lyrs.Activation('relu'))
model.add(lyrs.Dense(250))
model.add(lyrs.Activation('relu'))
model.add(lyrs.Dense(1))
model.add(lyrs.Activation('linear'))
model.compile(optimizer="adam", loss='mse')

In [12]:
history = model.fit(train, 
          train_target, 
          epochs=100,
          batch_size = len(train), 
          verbose=1, 
          validation_data=(test, test_target))

Epoch 1/100
1/1 [==============================] - 0s 106ms/step - loss: 0.9969 - val_loss: 115.7231
Epoch 2/100
1/1 [==============================] - 0s 16ms/step - loss: 126.1223 - val_loss: 8.0574
Epoch 3/100
1/1 [==============================] - 0s 20ms/step - loss: 9.7416 - val_loss: 4.3225
Epoch 4/100
1/1 [==============================] - 0s 21ms/step - loss: 3.9491 - val_loss: 7.2750
Epoch 5/100
1/1 [==============================] - 0s 20ms/step - loss: 7.5213 - val_loss: 3.2692
Epoch 6/100
1/1 [==============================] - 0s 17ms/step - loss: 3.5241 - val_loss: 0.4380
Epoch 7/100
1/1 [==============================] - 0s 16ms/step - loss: 0.5082 - val_loss: 0.3768
Epoch 8/100
1/1 [==============================] - 0s 15ms/step - loss: 0.3686 - val_loss: 1.0889
Epoch 9/100
1/1 [==============================] - 0s 16ms/step - loss: 1.1423 - val_loss: 1.0360
Epoch 10/100
1/1 [==============================] - 0s 8ms/step - loss: 1.0709 - val_loss: 0.5283
Epoch 11/100
1/

Epoch 84/100
1/1 [==============================] - 0s 15ms/step - loss: 0.0695 - val_loss: 0.0747
Epoch 85/100
1/1 [==============================] - 0s 16ms/step - loss: 0.0690 - val_loss: 0.0746
Epoch 86/100
1/1 [==============================] - 0s 17ms/step - loss: 0.0677 - val_loss: 0.0746
Epoch 87/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0683 - val_loss: 0.0746
Epoch 88/100
1/1 [==============================] - 0s 14ms/step - loss: 0.0696 - val_loss: 0.0746
Epoch 89/100
1/1 [==============================] - 0s 16ms/step - loss: 0.0699 - val_loss: 0.0746
Epoch 90/100
1/1 [==============================] - 0s 14ms/step - loss: 0.0694 - val_loss: 0.0745
Epoch 91/100
1/1 [==============================] - 0s 16ms/step - loss: 0.0681 - val_loss: 0.0745
Epoch 92/100
1/1 [==============================] - 0s 15ms/step - loss: 0.0691 - val_loss: 0.0745
Epoch 93/100
1/1 [==============================] - 0s 16ms/step - loss: 0.0676 - val_loss: 0.0745
Epoch 94/1

In [13]:
predicted = model.predict(test)
balance = 1000.0
actions = 0.0

In [14]:
hist = [[1000, 0, 1000, 0, 90.99]]
for i in range(len(predicted)):
    previous = hist[len(hist)-1]
    if predicted[i][0] == 0:
        hist.append([previous[0], previous[1], previous[2], 0, prices[i+1] ]) # raw[1] = prices
    else:
        if predicted[i][0] > 0:  # we buy
            amount_in_order = previous[0]*predicted[i][0]
            balance = previous[0] - amount_in_order
            n_barrels = previous[1] + amount_in_order/prices[i+1]
        else: # we sell
            amount_in_order = previous[1]*(-predicted[i][0])
            balance = previous[0] + amount_in_order*prices[i+1]
            n_barrels = previous[1] - amount_in_order
        estimated_total_balance = balance + n_barrels*prices[i+1]
        hist.append([balance, n_barrels, estimated_total_balance, predicted[i][0], prices[i+1] ])

In [15]:
elem = hist[len(hist)-1]
print(elem[0]," | ", elem[1]," | ", elem[2]," | ", elem[3]," | ", elem[4])

364.4821371846848  |  6.842813874274995  |  964.1863451261454  |  0.004821902  |  87.64


In [19]:
# Le troisième élément du tableau correspond à la balance au bout de 10 ans
# Là encore, ce n'est pas très concluant
# La loss du modèle est de 0.07, il faudrait de 0.01 pour s'approcher d'un algorithme très performant